In [7]:
# Before calling the API, replace filename and ensure sdk is installed: "pip install unstructured-client"
# See https://docs.unstructured.io/api-reference/api-services/sdk for more details

import unstructured_client
from unstructured_client.models import operations, shared
from dotenv import find_dotenv, load_dotenv
import os
import json
import random
import chardet

In [ ]:
load_dotenv(find_dotenv())
UNSTRUCTURED_API_KEY = os.getenv("UNSTRUCTURED_API_KEY")
UNSTRUCTURED_API_URL = os.getenv("UNSTRUCTURED_API_URL")
LOCAL_FILE_INPUT_DIR = os.getenv("LOCAL_FILE_INPUT_DIR")
LOCAL_FILE_OUTPUT_DIR = os.getenv("LOCAL_FILE_OUTPUT_DIR")


print(UNSTRUCTURED_API_KEY, UNSTRUCTURED_API_URL)
print(LOCAL_FILE_INPUT_DIR, LOCAL_FILE_OUTPUT_DIR)

In [19]:
client = unstructured_client.UnstructuredClient(
    api_key_auth=UNSTRUCTURED_API_KEY,
    server_url=UNSTRUCTURED_API_URL,
)

filename = "test.pdf"
# with open(filename, "rb") as file:
#     raw_data = file.read()
#     detector = chardet.UniversalDetector()
#     detector.feed(raw_data)
#     detector.close()

# encoding = detector.result["encoding"]

with open(filename, "r", encoding="utf-8", errors="ignore") as file:
    data = file.read()


req = operations.PartitionRequest(
    partition_parameters=shared.PartitionParameters(
        files=shared.Files(
            content=data,
            file_name=filename,
            strategy="hi_res",
            extract_images_in_pdf=True,
            infer_table_structure=True,
        ),
        # --- Other partition parameters ---
        # Note: Defining 'strategy', 'chunking_strategy', and 'output_format'
        # parameters as strings is accepted, but will not pass strict type checking. It is
        # advised to use the defined enum classes as shown below.
        strategy=shared.Strategy.HI_RES,
        languages=["eng"],
        infer_table_structure=True,
        extract_element_types=["Table"],
        extract_image_block_types=["Image", "Table"],
        extract_images_in_pdf=True,
    ),
)

In [20]:
output = []

try:
    res = client.general.partition(request=req)
    for element in res.elements:
        # print(element)

        output.append(element)

    # print(output)
except Exception as e:
    print(e)

INFO: HTTP Request: POST https://api.unstructuredapp.io/general/v0/general "HTTP/1.1 500 Internal Server Error"
INFO: Failed to process a request due to API server error with status code 500. Attempting retry number 1 after sleep.
INFO: Server message - {"detail":"Cannot find Root object in pdf"}
ERROR: Failed to partition the document.
ERROR: Server responded with 500 - {"detail":"Cannot find Root object in pdf"}


{"detail":"Cannot find Root object in pdf"}


In [ ]:
output = str(output)
output = output.encode("utf8").decode("ascii", "ignore")


file = f"03_unstructured_{random.randint(1000,9999)}.json"


with open(file, "w") as json_file:


    # Write the array of dictionaries to the JSON file

    json.dump(output, json_file)